In [119]:
import pandas as pd
import polars as pl

from mintalib.samples import sample_prices
from mintalib.indicators import SMA, EMA, MACD, RSI


In [120]:
prices = sample_prices()
plprices = pl.from_dataframe(prices.reset_index())
plprices

date,open,high,low,close,volume
datetime[ns],f64,f64,f64,f64,i64
1980-12-12 00:00:00,0.098943,0.099373,0.098943,0.098943,469033600
1980-12-15 00:00:00,0.094211,0.094211,0.093781,0.093781,175884800
1980-12-16 00:00:00,0.087328,0.087328,0.086898,0.086898,105728000
1980-12-17 00:00:00,0.089049,0.089479,0.089049,0.089049,86441600
1980-12-18 00:00:00,0.09163,0.092061,0.09163,0.09163,73449600
…,…,…,…,…,…
2024-10-15 00:00:00,233.610001,237.490005,232.369995,233.850006,64751400
2024-10-16 00:00:00,231.600006,232.119995,229.839996,231.779999,34082200
2024-10-17 00:00:00,233.429993,233.850006,230.520004,232.149994,32993800


In [121]:
import functools

from abc import ABCMeta, abstractmethod


class Study(metaclass=ABCMeta):
    """callable/chainable with process method and composition"""

    __pandas_priority__ = 5000

    @abstractmethod
    def __call__(self, prices): ...

    def __or__(self, other):
        """pipe into callable"""

        if not callable(other):
            return NotImplemented

        return self.pipe(other)


    def pipe(self, func, **kwargs):
        """pipe into callable with optional arguments"""

        if kwargs:
            func = functools.partial(func, **kwargs)

        return ChainedStudy(self, func)


class ChainedStudy(Study):
    """chain of callables/studies"""

    funcs: tuple = ()

    def __init__(self, *funcs):
        for func in funcs:
            if not callable(func):
                raise TypeError(f"Argument {func!r} is not callable!")
        self.funcs = funcs

    def __repr__(self):
        return " | ".join(repr(fn) for fn in self.funcs)

    def __call__(self, prices):
        result = prices
        
        for func in self.funcs:
            if result is None:
                return
            result = func(result)

        return result

    def pipe(self, func, **kwargs):
        """pipe into callable with optional arguments"""

        if kwargs:
            func = functools.partial(func, **kwargs)

        funcs = self.funcs + (func,)
        return self.__class__(*funcs)





In [122]:

class QuickStudy(Study):
    """Update Study"""

    args: tuple = ()
    kwargs: dict = {}

    def items(self):
        for arg in self.args:
            yield None, arg
        for kv in self.kwargs.items():
            yield kv
        
    def __init__(self, *args, **kwargs):
        self.args = args
        self.kwargs = kwargs


    def __repr__(self):
        cname = self.__class__.__name__
        params = ", ".join(f"{k}={v!r}" if k else repr(v) for k, v in self.items())
        return f"{cname}({params})"


    def __call__(self, prices):
        if not hasattr(prices, 'columns'):
            raise ValueError("DataFrame expected!")

        backend = getattr(prices, '__module__', None).partition('.')[0]

        if backend == "pandas":
            return self.apply_pandas(prices)
        
        if backend == "polars":
            return self.apply_polars(prices)
        
        raise ValueError(f"Unsupported DataFrame type: {backend}")
       
    
    def apply_pandas(self, prices):
        import pandas as pd

        columns = dict()
        
        for name, func in self.items():
            result = func(prices)

            if hasattr(result, 'columns'):
                columns.update(result)
            elif name is not None:
                columns[name] = result
            elif hasattr(result, 'name'):
                columns[result.name] = result
            else:
                raise ValueError(f"Unexpected result type {type(result)!r} in positional args!")
                
        return pd.DataFrame(columns, index=prices.index)


    def apply_polars(self, prices):
        import polars as pl

        columns = dict()

        for name, func in self.items():
            result = func(prices)

            if hasattr(result, 'columns'):
                columns.update(result.to_dict())
            elif name is not None:
                columns[name] = result
            elif hasattr(result, 'name'):
                columns[result.name] = result
            else:
                raise ValueError(f"Unexpected result type {type(result)!r} in positional args!")

        return pl.DataFrame(columns)


study = QuickStudy(MACD(), sma20 = SMA(20), sma50 = SMA(50) )
study



QuickStudy(MACD(12, 26, 9), sma20=SMA(20), sma50=SMA(50))

In [123]:
prices.pipe(study)

,macd,macdsignal,macdhist,sma20,sma50
date,,,,,
1980-12-12,NaN,NaN,NaN,NaN,NaN
1980-12-15,NaN,NaN,NaN,NaN,NaN
1980-12-16,NaN,NaN,NaN,NaN,NaN
1980-12-17,NaN,NaN,NaN,NaN,NaN
1980-12-18,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...
2024-10-15,1.815958,1.313965,0.501993,227.524000,224.138625
2024-10-16,1.941114,1.439395,0.501719,228.078500,224.634417
2024-10-17,2.046565,1.560829,0.485736,228.242500,225.085868


In [124]:
plprices.pipe(study)

macd,macdsignal,macdhist,sma20,sma50
f64,f64,f64,f64,f64
null,null,null,null,null
null,null,null,null,null
null,null,null,null,null
null,null,null,null,null
null,null,null,null,null
…,…,…,…,…
1.815958,1.313965,0.501993,227.524,224.138625
1.941114,1.439395,0.501719,228.0785,224.634417
2.046565,1.560829,0.485736,228.2425,225.085868
